# Notebook Imports

In [1]:
%pip install ultralytics
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch torchvision
%pip install opencv-python  # For video processing

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [3]:
import cv2
import time
import os
import numpy as np
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt

from ultralytics import YOLO

from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights, fasterrcnn_resnet50_fpn, maskrcnn_resnet50_fpn, ssd300_vgg16


### Video Details 

In [4]:
VIDEO = 'video for project.mp4'
 
# Save the output video file
OUTPUT_VIDEO_YOLO_DET = 'video_obj_detect_yolo.mp4'
OUTPUT_VIDEO_YOLO_SEG = 'video_obj_segementation_yolo.mp4'
OUTPUT_VIDEO_FASTER_RCNN_DET = 'video_obj_detect_faster_rcnn.mp4'
OUTPUT_VIDEO_FASTER_RCNN_SEG = 'video_obj_segmentation_faster_rcnn.mp4'
OUTPUT_VIDEO_SSD_DET = 'video_obj_detect_ssd.mp4'
OUTPUT_VIDEO_SSD_SEG = 'video_obj_segmentation_ssd.mp4'

FONT = cv2.FONT_HERSHEY_SIMPLEX

COCO_NAMES = ["person" , "bicycle" , "car" , "motorcycle" , "airplane" , "bus" , "train" , "truck" , "boat" , "traffic light" , "fire hydrant" , "street sign" , 
              "stop sign" , "parking meter" , "bench" , "bird" , "cat" , "dog" , "horse" , "sheep" , "cow" , "elephant" , "bear" , "zebra" , "giraffe" , "hat" , "backpack" , 
              "umbrella" , "shoe" , "eye glasses" , "handbag" , "tie" , "suitcase" , "frisbee" , "skis" , "snowboard" , "sports ball" , "kite" , "baseball bat" , "baseball glove" , 
              "skateboard" , "surfboard" , "tennis racket" , "bottle" , "plate" , "wine glass" , "cup" , "fork" , "knife" , "spoon" , "bowl" , "banana" , "apple" , "sandwich" , 
              "orange" , "broccoli" , "carrot" , "hot dog" ,"pizza" , "donut" , "cake" , "chair" , "couch" , "potted plant" , "bed" ,"mirror" , "dining table" , "window" , "desk" , 
              "toilet" , "door" , "tv" , "laptop" , "mouse" , "remote" , "keyboard" , "cell phone" , "microwave" ,"oven" , "toaster" , "sink" , "refrigerator" , "blender" , "book" ,
              "clock" , "vase" , "scissors" , "teddy bear" , "hair drier" , "toothbrush" , "hair brush"]

# YOLO8 Model

In [5]:
def run_model(model, video, output_video):
    model = model
    cap = cv2.VideoCapture(video)

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    # Get frame width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_video, fourcc, 20.0, (frame_width, frame_height))

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            print("No frame...")
            break

        # Predict on image
        results = model.track(source=frame, persist=True, tracker='bytetrack.yaml')
        frame = results[0].plot()

        # Write the frame to the output video file
        out.write(frame)

        # Display the resulting frame
        cv2.imshow("ObjectDetection", frame)

        # Terminate run when "Q" pressed
        if cv2.waitKey(1) == ord("q"):
            break

    # When everything done, release the capture
    cap.release()

    # Release the video recording
    # out.release()
    cv2.destroyAllWindows()

# Object Detection
run_model(model=YOLO('yolov8n.pt', "v8"), video=VIDEO, output_video=OUTPUT_VIDEO_YOLO_DET)

# Object Segmentation
run_model(model=YOLO('yolov8n-seg.pt', "v8"), video=VIDEO, output_video=OUTPUT_VIDEO_YOLO_SEG)


0: 384x640 4 persons, 1 car, 163.5ms
Speed: 21.7ms preprocess, 163.5ms inference, 15.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 car, 145.8ms
Speed: 0.0ms preprocess, 145.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 car, 151.1ms
Speed: 3.4ms preprocess, 151.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 2 cars, 119.3ms
Speed: 2.7ms preprocess, 119.3ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 car, 103.6ms
Speed: 0.0ms preprocess, 103.6ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 2 cars, 125.0ms
Speed: 0.0ms preprocess, 125.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 2 cars, 78.1ms
Speed: 15.6ms preprocess, 78.1ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 2 cars, 109.7ms
Spe

# Faster R-CNN

### Object Detection

In [6]:
def get_model():
    # Load a pre-trained Faster R-CNN model    
    weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    model = fasterrcnn_resnet50_fpn(weights=weights, pretrained=True)
    model.eval()
    return model

def faster_rcnn_object_detection(model, frame):
    # Transform frame to tensor and add batch dimension
    transform = T.Compose([T.ToTensor()])
    frame_tensor = transform(frame).unsqueeze(0)

    with torch.no_grad():
        prediction = model(frame_tensor)

    bboxes, labels, scores = prediction[0]["boxes"], prediction[0]["labels"], prediction[0]["scores"]

    # num = torch.argwhere(scores > 0.9).shape[0]

    # Draw boxes and labels on the frame
    for i in range(len(prediction[0]['boxes'])):
        xmin, ymin, xmax, ymax = bboxes[i].numpy().astype('int')
        class_name = COCO_NAMES[labels.numpy()[i] -1]
        
        if scores[i] > 0.9:  # Only draw boxes for confident predictions
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 3)
            
            # Put label
            label = f"{class_name}: {scores[i]:.2f}"
            cv2.putText(frame, label, (xmin, ymin - 10), FONT, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
    
    return frame

# Set up the model
model = get_model()

# Video capture setup
cap = cv2.VideoCapture(VIDEO)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Get frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(OUTPUT_VIDEO_FASTER_RCNN_DET, fourcc, 20.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("No frame...")
        break

    # Process frame
    processed_frame = faster_rcnn_object_detection(model, frame)
    
    # Write the processed frame to output
    out.write(processed_frame)
    
    # Display the frame
    cv2.imshow('Frame', processed_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything is finished
cap.release()
out.release()
cv2.destroyAllWindows()

No frame...


### Object Segementation

In [16]:
# Load the pre-trained Mask R-CNN model
model = maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Function to overlay masks and draw rectangles and labels on the frame
def faster_rcnn_object_segmentation(frame, threshold=0.9):
    # Function to preprocess the frame
    transform = T.Compose([T.ToTensor()])
    frame_tensor = transform(frame).unsqueeze(0)

    with torch.no_grad():
        predictions = model(frame_tensor)
    
    labels = predictions[0]['labels'].cpu().numpy()
    masks = predictions[0]['masks'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    boxes = predictions[0]['boxes'].cpu().numpy()
    
    overlay = frame.copy()
    
    for i in range(len(masks)):
        if scores[i] > threshold:
            mask = masks[i, 0]
            mask = (mask > 0.6).astype(np.uint8)
            color = np.random.randint(0, 255, (3,), dtype=np.uint8)
            overlay[mask == 1] = frame[mask == 1] * 0.5 + color * 0.5

            xmin, ymin, xmax, ymax = boxes[i].astype('int')
            class_name = COCO_NAMES[labels[i] - 1]
            
            # Draw rectangle
            cv2.rectangle(overlay, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            
            # Put label
            label = f"{class_name}: {scores[i]:.2f}"
            cv2.putText(overlay, label, (xmin, ymin - 10), FONT, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
    
    return overlay

# Capture video
cap = cv2.VideoCapture(VIDEO)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Get frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(OUTPUT_VIDEO_FASTER_RCNN_SEG, fourcc, 20.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("No frame...")
        break

    # Overlay masks
    processed_frame = faster_rcnn_object_segmentation(frame)

    # Write the processed frame to output
    out.write(processed_frame)
    
    # Display the frame
    cv2.imshow('Frame', processed_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything is finished
cap.release()
out.release()
cv2.destroyAllWindows()

No frame...


# SSD (Single Shot Detection)

### Object Detection

In [8]:
# Load the pre-trained SSD model
model = ssd300_vgg16(pretrained=True)
model.eval() 

def ssd_object_detection(frame, threshold=0.5):
    # Function to preprocess the frame
    transform = T.Compose([T.ToTensor()])
    frame_tensor = transform(frame).unsqueeze(0)

    with torch.no_grad():
        predictions = model(frame_tensor)
    
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    boxes = predictions[0]['boxes'].cpu().numpy()

    
    for i in range(len(boxes)):
        if scores[i] > threshold:

            xmin, ymin, xmax, ymax = boxes[i].astype('int')
            class_name = COCO_NAMES[labels[i] - 1]
            
            # Draw rectangle
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            
            # Put label
            label = f"{class_name}: {scores[i]:.2f}"
            cv2.putText(frame, label, (xmin, ymin - 10), FONT, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
    
    return frame

# Capture video
cap = cv2.VideoCapture(VIDEO)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Get frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(OUTPUT_VIDEO_SSD_DET, fourcc, 20.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("No frame...")
        break

    # Overlay masks
    processed_frame = ssd_object_detection(frame)

    # Write the processed frame to output
    out.write(processed_frame)
    
    # Display the frame
    cv2.imshow('Frame', processed_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything is finished
cap.release()
out.release()
cv2.destroyAllWindows()

C:\Users\fatim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


No frame...


### Object Segmentation

In [9]:
# Load the pre-trained SSD model
model = ssd300_vgg16(pretrained=True)
model.eval()

def ssd_object_segmentation(frame, threshold=0.5):
    # Function to preprocess the frame
    transform = T.Compose([T.ToTensor()])
    frame_tensor = transform(frame).unsqueeze(0)

    with torch.no_grad():
        predictions = model(frame_tensor)

    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    boxes = predictions[0]['boxes'].cpu().numpy()

    for i in range(len(boxes)):
        if scores[i] > threshold:
            xmin, ymin, xmax, ymax = boxes[i].astype('int')
            class_name = COCO_NAMES[labels[i] - 1]

            # Extract the detected object from the frame
            object_segment = frame[ymin:ymax, xmin:xmax]

            # Convert to grayscale and threshold to create a mask
            gray = cv2.cvtColor(object_segment, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

            # Find contours
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw the contours on the original frame
            cv2.drawContours(frame[ymin:ymax, xmin:xmax], contours, -1, (0, 255, 0), thickness=cv2.FILLED)

            # Put label above the box
            label = f"{class_name}: {scores[i]:.2f}"
            cv2.putText(frame, label, (xmin, ymin - 10), FONT, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
    
    return frame


# Capture video
cap = cv2.VideoCapture(VIDEO)  # replace with actual video file path

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Get frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(OUTPUT_VIDEO_SSD_SEG, fourcc, 20.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("No frame...")
        break

    # Overlay segmentation masks
    processed_frame = ssd_object_segmentation(frame)

    # Write the processed frame to output
    out.write(processed_frame)
    
    # Display the frame
    cv2.imshow('Frame', processed_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything once finished
cap.release()
out.release()
cv2.destroyAllWindows()

No frame...


# Evaluation

In [10]:
# MODELS
yolo_model = YOLO('yolov8n.pt', "v8")
faster_rccn_model = maskrcnn_resnet50_fpn(pretrained=True).eval()
ssd_model = ssd300_vgg16(pretrained=True).eval()

### Inference Time (s)

In [11]:
def preprocess_frame(frame, model_type):
    if model_type == 'yolo':
        return frame  # Assuming YOLO model can take the frame directly
    else:
        transform = T.Compose([
            T.ToPILImage(),
            T.Resize((300, 300)),
            T.ToTensor()
        ])
        return transform(frame)


def measure_performance(model, frames, model_type):
    times = []
    for frame in frames:
        frame_tensor = preprocess_frame(frame, model_type)
        if model_type != 'yolo':
            frame_tensor = frame_tensor.unsqueeze(0)  # Add batch dimension

        start = time.time()
        with torch.no_grad():
            if model_type == 'yolo':
                results = model(frame)
            else:
                results = model(frame_tensor)
        end = time.time()
        times.append(end - start)
    avg_time = sum(times) / len(times)
    return avg_time, results

In [12]:
# Measure inference time on a sample of frames from the video
cap = cv2.VideoCapture(VIDEO)
sample_frames = []
for i in range(20):  # Change 10 to the number of frames you want to sample
    ret, frame = cap.read()
    if not ret:
        break
    sample_frames.append(frame)
cap.release()

# Measure performance
models = [yolo_model, faster_rccn_model, ssd_model]
types = ['yolo', 'faster_rccn', 'ssd']

for i in range(len(models)):
    model_time, model_results = measure_performance(model=models[i], frames=sample_frames, model_type=types[i])
    print(f'{types[i].upper()} Detection Inference Time: {model_time:.2f} seconds per frame')


0: 384x640 8 persons, 3 cars, 117.4ms
Speed: 0.0ms preprocess, 117.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 cars, 127.0ms
Speed: 2.0ms preprocess, 127.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 156.5ms
Speed: 0.0ms preprocess, 156.5ms inference, 9.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 159.8ms
Speed: 0.0ms preprocess, 159.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 3 cars, 139.9ms
Speed: 0.0ms preprocess, 139.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 cars, 147.5ms
Speed: 0.0ms preprocess, 147.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 116.0ms
Speed: 0.0ms preprocess, 116.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 cars, 116.6ms
Sp

### FPS (frames per second)

In [13]:
def calculate_fps(model, video, model_type, num_frames=10):
    cap = cv2.VideoCapture(video)
    if not cap.isOpened():
        print("Cannot open video")
        return None

    frame_count = 0
    total_time = 0

    while frame_count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame_tensor = preprocess_frame(frame, model_type)
        if model_type != 'yolo':
            frame_tensor = frame_tensor.unsqueeze(0)  # Add batch dimension

        start_time = time.time()
        with torch.no_grad():
            if model_type == 'yolo':
                results = model(frame)
            else:
                results = model(frame_tensor)
        end_time = time.time()
        total_time += (end_time - start_time)
        frame_count += 1

    cap.release()

    if frame_count > 0:
        fps = frame_count / total_time
        return fps
    else:
        return None

In [14]:
# Calculate FPS
models = [yolo_model, faster_rccn_model, ssd_model]
types = ['yolo', 'faster rccn', 'ssd']

for i in range(len(models)):    
    fps = calculate_fps(model=models[i], video=VIDEO, num_frames=10, model_type=types[i])
    if fps:
        print(f'{types[i].upper()} Detection FPS: {fps:.2f} frames per second')
    else:
        print('Could not calculate FPS')


0: 384x640 8 persons, 3 cars, 120.1ms
Speed: 0.0ms preprocess, 120.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 cars, 117.1ms
Speed: 5.0ms preprocess, 117.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 137.9ms
Speed: 3.1ms preprocess, 137.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 117.9ms
Speed: 15.6ms preprocess, 117.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 3 cars, 127.9ms
Speed: 0.9ms preprocess, 127.9ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 cars, 120.9ms
Speed: 3.4ms preprocess, 120.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 107.3ms
Speed: 0.0ms preprocess, 107.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 cars, 108.4ms
S

### Model Size

In [15]:
# Load the models
faster_rcnn_model = fasterrcnn_resnet50_fpn(pretrained=True)
ssd_model = ssd300_vgg16(pretrained=True)
yolo_model = YOLO('yolov8n.pt').export()

# Save the models
torch.save(faster_rcnn_model.state_dict(), 'faster_rcnn_model.pth')
torch.save(ssd_model.state_dict(), 'ssd_model.pth')

# Get the file sizes
yolo_size = os.path.getsize('yolov8n.torchscript') / (1024 * 1024)
faster_rcnn_size = os.path.getsize('faster_rcnn_model.pth') / (1024 * 1024)  # in MB
ssd_size = os.path.getsize('ssd_model.pth') / (1024 * 1024)  # in MB

# Print
print(f"YOLO model size: {yolo_size:.2f} MB")
print(f"Faster R-CNN model size: {faster_rcnn_size:.2f} MB")
print(f"SSD model size: {ssd_size:.2f} MB")

C:\Users\fatim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Ultralytics YOLOv8.1.7 🚀 Python-3.11.9 torch-2.1.2+cpu CPU (Intel Core(TM) i5-10300H 2.50GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

TorchScript: starting export with torch 2.1.2+cpu...
TorchScript: export success ✅ 3.8s, saved as 'yolov8n.torchscript' (12.5 MB)

Export complete (6.3s)
Results saved to C:\ML
Predict:         yolo predict task=detect model=yolov8n.torchscript imgsz=640  
Validate:        yolo val task=detect model=yolov8n.torchscript imgsz=640 data=coco.yaml  
Visualize:       https://netron.app
YOLO model size: 12.47 MB
Faster R-CNN model size: 159.79 MB
SSD model size: 135.99 MB
